# Welcome
Before start, make sure that you choose
* Runtime Type = Python 3
* Hardware Accelerator = GPU

In the **Runtime** menu -> **Change runtime type**

This Google Colab notebook provides a comprehensive, multi-step pipeline for upscaling and refining image datasets using the powerful Real-ESRGAN model.

Key Features:
- Batch Processing & Organization: Upscales images from multiple subfolders in one go, preserving your original folder structure in the output.
- VRAM Optimization: Features a configurable Tiling mechanism (recommended $1024 \text{ px}$ for T4 GPUs) to prevent Out-of-Memory (OOM) errors and significantly speed up processing of large images.
- Smart Cropping: Allows selection of multiple target aspect ratios (e.g., $1:1, 4:3, 16:9$). The script automatically crops each image to the closest matching selected ratio to minimize content loss.
- Quality Control & Cleanup: Includes options for zero-padded file renaming, format conversion (e.g., PNG to JPG), optional final rescaling, and a cleanup step to remove the _out suffix for cleaner final file names.
- Final Delivery: Features a final step to automatically batch-zip each resulting output subfolder for easy download.

In [ ]:
# @markdown 🚀 1. Setup Environment and Install Dependencies (Click Run Only!)

import os
import subprocess
import shutil
import fileinput
import sys
import glob

# --- 1. Clean up and Clone Real-ESRGAN ---
print("Starting cleanup and cloning...")

# OPTIMIZATION: Remove aggressive uninstall of torch/torchvision.
# Only uninstall the project-specific dependencies that might conflict.
!pip uninstall basicsr facexlib gfpgan -y

# Clone Real-ESRGAN and navigate to the directory
!git clone https://github.com/xinntao/Real-ESRGAN.git
%cd Real-ESRGAN

# --- 2. Install Real-ESRGAN's Specific Dependencies ---
# We are now relying on Colab's pre-installed and optimized PyTorch/Torchvision stack.
print("Installing Real-ESRGAN dependencies (relying on pre-installed PyTorch)...")
!pip install basicsr facexlib gfpgan
!pip install -r requirements.txt
!python setup.py develop
print("Dependencies installed.")


# --- 3. CRITICAL: PATCH THE BASICSr SOURCE CODE ---
# This necessary patch remains, as it fixes the dependency compatibility issue with modern PyTorch.

# The target file path is within the installed site-packages directory
try:
    target_file_path = glob.glob('/usr/local/lib/python*/dist-packages/basicsr/data/degradations.py')[0]
except IndexError:
    print("ERROR: basicsr/data/degradations.py not found. Installation may have failed.")
    target_file_path = None # Set to None to skip patching

if target_file_path:
    print(f"\nPatching critical file: {target_file_path}")
    search_line = 'from torchvision.transforms.functional_tensor import rgb_to_grayscale'
    replace_line = 'from torchvision.transforms.functional import rgb_to_grayscale'

    try:
        with fileinput.FileInput(target_file_path, inplace=True) as file:
            for line in file:
                if search_line in line:
                    sys.stdout.write(replace_line + '\n')
                else:
                    sys.stdout.write(line)
        print("Patch successful! ModuleNotFoundError should be resolved.")

    except Exception as e:
        print(f"ERROR DURING PATCHING: {e}")


print("\nEnvironment setup complete. You can now run the next cells.")

Starting cleanup and cloning...
Found existing installation: basicsr 1.4.2
Uninstalling basicsr-1.4.2:
  Successfully uninstalled basicsr-1.4.2
Found existing installation: facexlib 0.3.0
Uninstalling facexlib-0.3.0:
  Successfully uninstalled facexlib-0.3.0
Found existing installation: gfpgan 1.3.8
Uninstalling gfpgan-1.3.8:
  Successfully uninstalled gfpgan-1.3.8
Cloning into 'Real-ESRGAN'...
remote: Enumerating objects: 759, done.
remote: Counting objects: 100% (121/121), done.
remote: Compressing objects: 100% (23/23), done.
remote: Total 759 (delta 106), reused 98 (delta 98), pack-reused 638 (from 1)
Receiving objects: 100% (759/759), 5.38 MiB | 12.10 MiB/s, done.
Resolving deltas: 100% (415/415), done.
/content/Real-ESRGAN/Real-ESRGAN
Installing Real-ESRGAN dependencies (relying on pre-installed PyTorch)...
  Using cached basicsr-1.4.2-py3-none-any.whl
  Using cached facexlib-0.3.0-py3-none-any.whl.metadata (4.6 kB)
  Using cached gfpgan-1.3.8-py3-none-any.whl.metadata (12 kB)
Us

# 2. Connect Google Drive

Configure the folder in Drive and ready to be processed by Real-ESRGAN

In [ ]:
import sys
import subprocess
import os
import cv2
from google.colab import files
import shutil
from google.colab import drive

# Mount the drive (REQUIRED to access /content/drive/MyDrive/...)
drive.mount('/content/drive')

# @markdown ### **Google Drive and Folder Configuration**
# @markdown Set the **PARENT FOLDERS**. Cell 3 will scan these folders for images/subfolders.

# --- Define Your Parent Paths ---
# The PARENT folder on Drive that CONTAINS all your sub-project folders (e.g., 'dataset/')
PARENT_INPUT_FOLDER = '/content/drive/MyDrive/dataset' # @param {type: "string"}
# The PARENT folder on Drive where results will be saved, maintaining the same structure (e.g., 'upscale_image/')
PARENT_OUTPUT_FOLDER = '/content/drive/MyDrive/upscale_image' # @param {type: "string"}

# @markdown ---
# @markdown ### 📝 Preprocessing Options: Multiple Upscale Folders

# @markdown Enter the names of the specific subfolders inside PARENT_INPUT_FOLDER (the dataset folder).
# @markdown **Separate multiple subfolder names with a comma.** (e.g., project_A, project_B, project_C)
TEMP_RENAME_SUBFOLDERS = "think_mark, ace_atorney_hands_up, ace_atorney_pointing" # @param {type: "string"}

# @markdown Toggle this option to enable file renaming. Disable it if you only want to upscale the images from dataset folder only
enable_file_renaming = False # @param {type:"boolean"}

# ----------------------------------------------------
# FILE RENAMING LOGIC (Applied to all specified subfolders)
# ----------------------------------------------------
if enable_file_renaming:
    # Split the input string into a list of subfolder names
    subfolder_names = [name.strip() for name in TEMP_RENAME_SUBFOLDERS.split(',') if name.strip()]

    if not subfolder_names:
        print("⚠️ Renaming is enabled, but no subfolder names were provided. Skipping renaming.")
    else:
        print("\nStarting batch file renaming process...")
        total_renamed_files = 0

        for subfolder_name in subfolder_names:
            rename_folder_path = os.path.join(PARENT_INPUT_FOLDER, subfolder_name)

            if not os.path.isdir(rename_folder_path):
                print(f"Error: Subfolder '{subfolder_name}' not found at {rename_folder_path}. Skipping.")
                continue

            # Get all files in the current directory, filtering for common image extensions
            image_extensions = ('.jpg', '.jpeg', '.png', '.webp', '.tif', '.tiff')
            files_to_rename = sorted([f for f in os.listdir(rename_folder_path) if f.lower().endswith(image_extensions)])

            if files_to_rename:
                files_renamed_in_folder = 0
                for i, old_filename in enumerate(files_to_rename):
                    base, ext = os.path.splitext(old_filename)
                    new_filename = str(i + 1) + ext.lower()

                    old_filepath = os.path.join(rename_folder_path, old_filename)
                    new_filepath = os.path.join(rename_folder_path, new_filename)

                    if old_filepath != new_filepath:
                        os.rename(old_filepath, new_filepath)
                        files_renamed_in_folder += 1

                total_renamed_files += files_renamed_in_folder
                print(f"✅ Renamed {files_renamed_in_folder} files in subfolder: '{subfolder_name}'")
            else:
                print(f"ℹ️ No images found for renaming in subfolder: '{subfolder_name}'")

        if total_renamed_files > 0:
            print(f"\n✨ Batch renaming complete. Total files processed: {total_renamed_files}.")
        else:
            print("\nRenaming finished, but no files were modified.")

else:
    print("File renaming is DISABLED.")
# ----------------------------------------------------

print(f"\nParent Input Directory set to: {PARENT_INPUT_FOLDER}")
print(f"Parent Output Directory set to: {PARENT_OUTPUT_FOLDER}")
print("Ready for processing in Cell 3.")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
File renaming is DISABLED.

Parent Input Directory set to: /content/drive/MyDrive/dataset
Parent Output Directory set to: /content/drive/MyDrive/upscale_image
Ready for processing in Cell 3.


# 3. Upscale

Upload the images to be processed by Real-ESRGAN

In [ ]:
# Execute Real-ESRGAN inference

import os
import glob
import shutil

# NOTE: PARENT_INPUT_FOLDER and PARENT_OUTPUT_FOLDER are now defined in Cell 2.

# @markdown ### ⚙️ Upscaler Configuration
# @markdown Choose the standard Real-ESRGAN model to use.

upscaler_name = "RealESRGAN_x4plus" # @param ["RealESRGAN_x4plus", "RealESRGAN_x4plus_anime_6B"]

# @markdown ---
# @markdown ### 📂 Folder Processing Mode
# @markdown By default, images in the root of the input folder are processed.
# @markdown Toggle ON to process images found in **ALL SUBFOLDERS** instead.
process_subfolders = True # @param {type:"boolean"}

# @markdown ---
# @markdown ### 🖼️ Processing Options
# @markdown Adjust the scale factor and toggle face enhancement.

scale_factor = 3.5 # @param {type:"number"}

enable_face_enhance = False # @param {type:"boolean"}

# @markdown ---
# @markdown ### 📉 VRAM Optimization (Tiling)
# @markdown For T4 colab a range between 512-1024 is recommended. The higher the faster process, but use more VRAM.
# @markdown Set to 0 or leave blank to disable tiling (High VRAM risk).
tile_size = 512 # @param {type:"number"}


# Define flags dynamically
face_enhance_flag = '--face_enhance' if enable_face_enhance else ''
# If tile_size is set and greater than 0, add the tiling flag
tile_flag = f'-t {tile_size}' if tile_size > 0 else ''


# --- FIND ALL INPUT FOLDERS TO PROCESS BASED ON TOGGLE ---

if process_subfolders:
    # MODE 1: Process subfolders (Input: PARENT_INPUT_FOLDER/Subfolder1)
    print("Mode: Processing all subfolders.")
    input_dirs = sorted([d for d in glob.glob(os.path.join(PARENT_INPUT_FOLDER, '*')) if os.path.isdir(d)])

    if not input_dirs:
        print(f"Warning: Toggle is ON, but no subfolders were found in {PARENT_INPUT_FOLDER}.")
        print("Falling back to processing the parent folder directly.")
        input_dirs = [PARENT_INPUT_FOLDER]

else:
    # MODE 2: Process the parent folder directly (Input: PARENT_INPUT_FOLDER)
    print("Mode: Processing images in the root of the parent folder.")
    input_dirs = [PARENT_INPUT_FOLDER]


# --- PROCESS EACH INPUT LOCATION ---
total_processed_locations = 0
for input_dir in input_dirs:

    # 1. Determine the output path
    if process_subfolders:
        # Output goes into a subfolder matching the input subfolder name
        subfolder_name = os.path.basename(input_dir)
        output_dir = os.path.join(PARENT_OUTPUT_FOLDER, subfolder_name)
    else:
        # Output goes directly into the parent output folder
        output_dir = PARENT_OUTPUT_FOLDER

    # 2. Clean up and ensure the output directory exists
    if os.path.isdir(output_dir):
        # We clean the output folder for the specific location being processed
        print(f"Cleaning previous results in: {output_dir}")
        shutil.rmtree(output_dir)
    os.makedirs(output_dir, exist_ok=True)

    print("-" * 50)
    print(f"Input: {input_dir}")
    print(f"Output: {output_dir}")
    print(f"Tiling: {'Enabled' if tile_size > 0 else 'Disabled'}")

    # 3. Construct the full command
    command = f"""python inference_realesrgan.py -n {upscaler_name} \
      -i "{input_dir}" \
      -o "{output_dir}" \
      --outscale {scale_factor} \
      {face_enhance_flag} \
      {tile_flag}
    """

    # 4. Execute the command
    !{command}

    total_processed_locations += 1

print("=" * 50)
print(f"✅ Upscaling complete! Processed {total_processed_locations} input location(s).")
print("Files saved to the output folder structure, ready for format conversion and resizing in the next cell.")

Mode: Processing all subfolders.
Cleaning previous results in: /content/drive/MyDrive/upscale_image/ace_atorney_hands_up
--------------------------------------------------
Input: /content/drive/MyDrive/dataset/ace_atorney_hands_up
Output: /content/drive/MyDrive/upscale_image/ace_atorney_hands_up
Tiling: Enabled
Testing 0 20408fa684536fb223db3321a5694004
	Tile 1/2
	Tile 2/2
Testing 1 __chihiro_blue_archive_and_1_more_drawn_by_generic_33__sample-847704045843ea04c4b6f3e1e70e8c8b
	Tile 1/4
	Tile 2/4
	Tile 3/4
	Tile 4/4
Testing 2 __dante_and_miles_edgeworth_ace_attorney_and_2_more_drawn_by_denaseey__sample-4c2e7f827d13432d703eae0de0ef5ba3
	Tile 1/4
	Tile 2/4
	Tile 3/4
	Tile 4/4
Testing 3 __frisk_and_sans_utdr_and_2_more_drawn_by_hanbaobaobaozhu__sample-859b513b6940f29a71a8606caac46045
	Tile 1/2
	Tile 2/2
Testing 4 __hikawa_sayo_hikawa_hina_phoenix_wright_and_miles_edgeworth_bang_dream_and_1_more_drawn_by_fries_vanquisher__sample-12ff70e4facfb3c3e06631dfd2af3763
	Tile 1/2
	Tile 2/2
Testing 5

# 4. Rescale and change output format


In [ ]:
import cv2
import os
import glob
import numpy as np

# @markdown ### ⚙️ Resolution Rescaling Configuration
# @markdown Toggle this switch to enable or disable the resizing step entirely. Disable it to change output format only.

enable_rescaling = True # @param {type:"boolean"}

# @markdown ---

# @markdown ### 💾 Output File Configuration
# @markdown Choose the desired output file extension.

output_format = "png" # @param ["png", "jpg", "webp", "jpeg"]
# @markdown Set the target pixel size for the chosen dimension.

TARGET_SIDE_PIXELS = 1024 # @param {type:"number"}

# @markdown Choose which dimension to anchor the rescaling on. The other dimension will be calculated proportionally.

dimension_to_scale = "Minor Side (Shorter)" # @param ["Minor Side (Shorter)", "Major Side (Longer)", "Width", "Height"]

# @markdown ---

# --- VARIABLES FROM PREVIOUS CELLS ---
# PARENT_OUTPUT_FOLDER is now imported from Cell 2, replacing the hardcoded path.
# output_format (e.g., "png", "jpg") is imported implicitly from Cell 3.
INPUT_FOLDER = PARENT_OUTPUT_FOLDER # Use the parent folder defined in Cell 2
FINAL_EXTENSION = "." + output_format.lower()
JPEG_QUALITY = 95 # Quality for JPG output (0-100)

# =============================================================
# 1. FILE CONVERSION LOGIC (Runs first, regardless of rescaling toggle)
# =============================================================
print(f"Starting format conversion to: {FINAL_EXTENSION}")

# 💡 FIX: Use recursive glob (**) to find files in all subfolders.
# We look for files ending with '_out' (Real-ESRGAN suffix) but accept any extension.
all_files = sorted(glob.glob(os.path.join(INPUT_FOLDER, '**', '*_out.*'), recursive=True))
converted_files = []

if not all_files:
    # 💡 Fallback: If no files with '_out' are found, check the root folder (for single file runs/SwinIR runs)
    all_files = sorted(glob.glob(os.path.join(INPUT_FOLDER, '*.*')))

if not all_files:
    print("❌ No upscaled files found in the output directory or its subfolders. Skipping Cell 4.")
else:
    for file_path in all_files:
        if os.path.isdir(file_path):
            continue

        base_name, current_ext = os.path.splitext(file_path)
        current_ext = current_ext.lower()
        new_filepath = base_name + FINAL_EXTENSION

        # Load image (required for saving in new format)
        img = cv2.imread(file_path)

        if img is None:
            print(f"Warning: Could not read image at {file_path}")
            continue

        # Save in the target format if needed (or if rescaling is enabled, we need the final format)
        if current_ext != FINAL_EXTENSION:
            try:
                params = []
                if FINAL_EXTENSION == ".jpg" or FINAL_EXTENSION == ".jpeg":
                    params.append(cv2.IMWRITE_JPEG_QUALITY)
                    params.append(JPEG_QUALITY)

                cv2.imwrite(new_filepath, img, params)

                # Delete the old file
                os.remove(file_path)

            except Exception as e:
                print(f"ERROR converting {os.path.basename(file_path)}: {e}")
                continue # Skip to next file if conversion fails

        # Use the final path for subsequent rescaling
        converted_files.append(new_filepath)

    print(f"✅ Format conversion complete. Processed {len(converted_files)} files.")
    print("-" * 30)


    # =============================================================
    # 2. RESIZING LOGIC (Runs only if enabled)
    # =============================================================
    if not enable_rescaling:
        print("Rescaling is DISABLED. Final files remain at upscaled resolution.")
    else:
        print(f"Starting rescaling process. Targeting the '{dimension_to_scale}' side to be {TARGET_SIDE_PIXELS} pixels.")

        for file_path in converted_files:
            # 1. Read the image (now guaranteed to be in the correct format)
            img = cv2.imread(file_path)
            if img is None:
                continue

            h, w, _ = img.shape

            # 2. Determine the reference side based on user selection
            if dimension_to_scale == "Width":
                reference_side = w
            elif dimension_to_scale == "Height":
                reference_side = h
            elif dimension_to_scale == "Minor Side (Shorter)":
                reference_side = min(w, h)
            elif dimension_to_scale == "Major Side (Longer)":
                reference_side = max(w, h)
            else:
                reference_side = 1 # Fallback to prevent division by zero

            # 3. Calculate the required scaling factor
            scale_factor = int(TARGET_SIDE_PIXELS) / reference_side

            # 4. Calculate new dimensions and resize
            new_w = int(w * scale_factor)
            new_h = int(h * scale_factor)

            # 5. Check if rescaling is necessary
            if abs(scale_factor - 1.0) > 0.001:
                # Use INTER_AREA for downscaling (Factor < 1), INTER_CUBIC for upscaling/near 1.
                interpolation_method = cv2.INTER_AREA if scale_factor < 1 else cv2.INTER_CUBIC
                resized_img = cv2.resize(img, (new_w, new_h), interpolation=interpolation_method)

                # 6. Save the image back, maintaining the correct format (e.g., .png)
                cv2.imwrite(file_path, resized_img)

                print(f"Rescaled {os.path.basename(file_path)} (Old: {w}x{h}). New size: ({new_w}x{new_h}).")
            else:
                print(f"{os.path.basename(file_path)} is already near the target resolution. Skipping resize.")

        print("\n✅ Rescaling process complete.")

Starting format conversion to: .png
✅ Format conversion complete. Processed 69 files.
------------------------------
Starting rescaling process. Targeting the 'Minor Side (Shorter)' side to be 1024 pixels.
Rescaled 20408fa684536fb223db3321a5694004_out.png (Old: 2219x1480). New size: (1535x1024).
Rescaled __chihiro_blue_archive_and_1_more_drawn_by_generic_33__sample-847704045843ea04c4b6f3e1e70e8c8b_out.png (Old: 2422x1816). New size: (1365x1024).
Rescaled __dante_and_miles_edgeworth_ace_attorney_and_2_more_drawn_by_denaseey__sample-4c2e7f827d13432d703eae0de0ef5ba3_out.png (Old: 2975x2233). New size: (1364x1024).
Rescaled __frisk_and_sans_utdr_and_2_more_drawn_by_hanbaobaobaozhu__sample-859b513b6940f29a71a8606caac46045_out.png (Old: 2975x1673). New size: (1820x1023).
Rescaled __hikawa_sayo_hikawa_hina_phoenix_wright_and_miles_edgeworth_bang_dream_and_1_more_drawn_by_fries_vanquisher__sample-12ff70e4facfb3c3e06631dfd2af3763_out.png (Old: 2975x1673). New size: (1820x1023).
Rescaled __koizu

#  🛠️ 5. Aspect Ratio Cropping


In [ ]:
import cv2
import os
import glob
import math

# @markdown ### 📐 Aspect Ratio Cropping Configuration

# @markdown Toggle ON to enable cropping to a standard aspect ratio.
enable_ratio_crop = True # @param {type:"boolean"}

# @markdown ---
# @markdown **Select the Target Aspect Ratios** (The image will be cropped to the closest matching checked ratio)

# Ratio W/H | Name
# ----------------------------------------------------------------------
crop_1_1 = True # @param {type:"boolean"}
crop_3_2 = True # @param {type:"boolean"}
crop_2_3 = True # @param {type:"boolean"}
crop_4_3 = True # @param {type:"boolean"}
crop_3_4 = True # @param {type:"boolean"}
crop_4_5 = False # @param {type:"boolean"}
crop_5_4 = True # @param {type:"boolean"}
crop_5_7 = False # @param {type:"boolean"}
crop_7_5 = False # @param {type:"boolean"}
crop_16_9 = True # @param {type:"boolean"}
crop_9_16 = True # @param {type:"boolean"}
crop_1_2 = False # @param {type:"boolean"}
crop_2_1 = False # @param {type:"boolean"}

# @markdown ---

# PARENT_OUTPUT_FOLDER is defined in Cell 2.
INPUT_FOLDER = PARENT_OUTPUT_FOLDER

# --- Define ALL Possible Ratios Statically ---
ALL_RATIOS = [
    ("1:1", 1/1, crop_1_1),
    ("3:2", 3/2, crop_3_2),
    ("2:3", 2/3, crop_2_3),
    ("4:3", 4/3, crop_4_3),
    ("3:4", 3/4, crop_3_4),
    ("4:5", 4/5, crop_4_5),
    ("5:4", 5/4, crop_5_4),
    ("5:7", 5/7, crop_5_7),
    ("7:5", 7/5, crop_7_5),
    ("16:9", 16/9, crop_16_9),
    ("9:16", 9/16, crop_9_16),
    ("1:2", 1/2, crop_1_2),
    ("2:1", 2/1, crop_2_1)
]


# --- Cropping Logic ---

if not enable_ratio_crop:
    print("Aspect Ratio Cropping is DISABLED.")
else:
    # 1. Build a list of selected ratios (filter based on the boolean variable)
    selected_ratios = [(label, value) for label, value, is_checked in ALL_RATIOS if is_checked]

    if not selected_ratios:
        print("⚠️ Cropping is enabled, but no aspect ratios were selected. Skipping.")

    else:
        # Print all selected ratios for confirmation
        print(f"Starting Aspect Ratio Cropping. Target Ratios Selected: {[label for label, value in selected_ratios]}")

        # Find all files recursively (same method as Cell 4)
        all_files = sorted(glob.glob(os.path.join(INPUT_FOLDER, '**', '*.*'), recursive=True))
        image_extensions = ('.jpg', '.jpeg', '.png', '.webp')
        image_files = [f for f in all_files if os.path.isfile(f) and f.lower().endswith(image_extensions)]

        if not image_files:
            print("❌ No image files found for cropping. Skipping.")

        for file_path in image_files:
            img = cv2.imread(file_path)
            if img is None:
                continue

            h, w, _ = img.shape
            current_ratio = w / h

            # --- CRITICAL LOGIC: Find the best matching target ratio ---
            best_match_ratio = None
            min_ratio_diff = float('inf')
            best_match_label = ""

            for target_ratio_label, target_ratio_value in selected_ratios:
                # Calculate the difference relative to the target ratio.
                diff = abs(current_ratio / target_ratio_value - 1)

                if diff < min_ratio_diff:
                    min_ratio_diff = diff
                    best_match_ratio = target_ratio_value
                    best_match_label = target_ratio_label

            if best_match_ratio is None:
                continue

            TARGET_RATIO = best_match_ratio

            # 2. Determine the new dimensions based on the best match
            if current_ratio > TARGET_RATIO:
                # Current image is wider than the target ratio (crop width)
                new_h = h
                new_w = math.floor(h * TARGET_RATIO)
            else:
                # Current image is taller than the target ratio (crop height)
                new_w = w
                new_h = math.floor(w / TARGET_RATIO)

            # 3. Calculate crop margins for a perfect center crop
            crop_w_margin = (w - new_w) // 2
            crop_h_margin = (h - new_h) // 2

            if crop_w_margin < 0 or crop_h_margin < 0:
                 print(f"Warning on {os.path.basename(file_path)}: Crop calculated dimensions were invalid. Skipping.")
                 continue

            # 4. Perform the centered crop
            cropped_img = img[crop_h_margin : h - crop_h_margin,
                              crop_w_margin : w - crop_w_margin]

            # 5. Save the cropped image back, overwriting the original file
            cv2.imwrite(file_path, cropped_img)

            print(f"Cropped {os.path.basename(file_path)} (Current: {w}x{h}) to ({cropped_img.shape[1]}x{cropped_img.shape[0]}) using {best_match_label}.")

        print("\n✅ Aspect Ratio Cropping complete.")

Starting Aspect Ratio Cropping. Target Ratios Selected: ['1:1', '3:2', '2:3', '4:3', '3:4', '5:4', '16:9', '9:16']
Cropped 20408fa684536fb223db3321a5694004_out.png (Current: 1535x1024) to (1535x1024) using 3:2.
Cropped __chihiro_blue_archive_and_1_more_drawn_by_generic_33__sample-847704045843ea04c4b6f3e1e70e8c8b_out.png (Current: 1365x1024) to (1365x1024) using 4:3.
Cropped __dante_and_miles_edgeworth_ace_attorney_and_2_more_drawn_by_denaseey__sample-4c2e7f827d13432d703eae0de0ef5ba3_out.png (Current: 1364x1024) to (1364x1024) using 4:3.
Cropped __frisk_and_sans_utdr_and_2_more_drawn_by_hanbaobaobaozhu__sample-859b513b6940f29a71a8606caac46045_out.png (Current: 1820x1023) to (1818x1023) using 16:9.
Cropped __hikawa_sayo_hikawa_hina_phoenix_wright_and_miles_edgeworth_bang_dream_and_1_more_drawn_by_fries_vanquisher__sample-12ff70e4facfb3c3e06631dfd2af3763_out.png (Current: 1820x1023) to (1818x1023) using 16:9.
Cropped __koizumi_itsuki_suzumiya_haruhi_no_yuuutsu_and_1_more_drawn_by_iwamoto_

# 🧹 6. Remove _out Suffix


In [ ]:
import os
import glob

# @markdown ### ⚙️ Output File Cleanup
# @markdown Toggle this ON to remove the standard Real-ESRGAN suffix `_out` from all final image files.
enable_suffix_removal = True # @param {type:"boolean"}

# PARENT_OUTPUT_FOLDER is defined in Cell 2.
INPUT_FOLDER = PARENT_OUTPUT_FOLDER

if not enable_suffix_removal:
    print("File suffix removal is DISABLED. Files will retain the '_out' suffix.")

else:
    print(f"Starting removal of '_out' suffix from files in: {INPUT_FOLDER}")

    # 1. Recursively find all image files that contain the '_out' suffix
    # We look for files ending with '_out.ext'
    search_pattern = os.path.join(INPUT_FOLDER, '**', '*_out.*')
    files_to_rename = sorted(glob.glob(search_pattern, recursive=True))

    if not files_to_rename:
        print("ℹ️ No files found with the '_out' suffix. Cleanup complete.")
    else:
        renamed_count = 0
        for old_filepath in files_to_rename:

            # Check to prevent errors if glob finds a directory
            if os.path.isdir(old_filepath):
                continue

            # Get the directory and the filename
            directory = os.path.dirname(old_filepath)
            old_filename = os.path.basename(old_filepath)

            # 2. Construct the new filename by removing '_out'
            # We assume the suffix is before the last extension (e.g., '.png')
            new_filename = old_filename.replace('_out.', '.', 1) # Replace only once

            new_filepath = os.path.join(directory, new_filename)

            # 3. Rename the file
            try:
                os.rename(old_filepath, new_filepath)
                # print(f"Renamed: {old_filename} -> {new_filename}")
                renamed_count += 1
            except Exception as e:
                print(f"❌ ERROR renaming {old_filename}: {e}")

        print(f"\n✅ Suffix cleanup complete. Renamed {renamed_count} files.")

Starting removal of '_out' suffix from files in: /content/drive/MyDrive/upscale_image

✅ Suffix cleanup complete. Renamed 69 files.


# 📦 7. Compress to zip file


In [ ]:
import os
import shutil
import glob

# NOTE: PARENT_OUTPUT_FOLDER is defined in Cell 2.
# We will use shutil.make_archive for robust, platform-independent zipping.

# @markdown ### ⚙️ Batch Zipping Configuration
# @markdown Toggle this switch to zip all processed output folders.

enable_zipping = True # @param {type:"boolean"}

# @markdown Toggle this switch to delete the individual subfolders after they have been successfully zipped to save Drive space.
cleanup_folders_after_zip = False # @param {type:"boolean"}

# @markdown ---

if not enable_zipping:
    print("Zipping is DISABLED. Output folders remain on Google Drive.")

else:
    print(f"Starting batch zipping process in: {PARENT_OUTPUT_FOLDER}")

    # 1. Identify all subfolders created in the PARENT_OUTPUT_FOLDER (Multi-Folder Mode)
    output_subfolders = sorted(
        [d for d in glob.glob(os.path.join(PARENT_OUTPUT_FOLDER, '*')) if os.path.isdir(d)]
    )

    # 2. Check if we need to process the root folder (Single-Folder Mode)
    # Checks if no subfolders were found AND if the root folder contains upscaled files
    root_files_exist = glob.glob(os.path.join(PARENT_OUTPUT_FOLDER, '*.*'))

    if not output_subfolders and root_files_exist:
        print("Processing root output folder (single-folder mode detected).")
        folders_to_zip = [PARENT_OUTPUT_FOLDER]
    elif output_subfolders:
        folders_to_zip = output_subfolders
    else:
        print("⚠️ No output files or folders were found in the output directory. Skipping zipping.")
        folders_to_zip = []

    if folders_to_zip:
        zipped_count = 0

        for folder_path in folders_to_zip:
            folder_name = os.path.basename(folder_path)

            # 3. Determine the base name and destination for the zip file
            zip_base_name = folder_name
            zip_target_dir = os.path.dirname(folder_path) # The directory where the zip file is placed

            # If zipping the root output folder, adjust the base name/target directory
            if folder_path == PARENT_OUTPUT_FOLDER:
                 zip_base_name = os.path.basename(PARENT_OUTPUT_FOLDER)
                 zip_target_dir = os.path.dirname(PARENT_OUTPUT_FOLDER)

            print(f"--- Zipping '{folder_name}' ---")

            try:
                # shutil.make_archive creates the zip file
                shutil.make_archive(
                    base_name=os.path.join(zip_target_dir, zip_base_name),
                    format='zip',
                    root_dir=os.path.dirname(folder_path), # Start search from the directory *above* the folder_path
                    base_dir=folder_name # Only include the contents of the folder_name
                )
                print(f"✅ Created {zip_base_name}.zip")
                zipped_count += 1

                # 4. Optional Cleanup
                if cleanup_folders_after_zip:
                    shutil.rmtree(folder_path)
                    print(f"🧹 Cleaned up source folder: {folder_name}")

            except Exception as e:
                print(f"❌ ERROR zipping {folder_name}: {e}")

        print(f"\n==================================================")
        print(f"📦 Batch Zipping Complete. {zipped_count} archive(s) created in your Drive.")
        print(f"Zip files are located in: {os.path.dirname(PARENT_OUTPUT_FOLDER)}/")

Starting batch zipping process in: /content/drive/MyDrive/upscale_image
--- Zipping 'ace_atorney_hands_up' ---
✅ Created ace_atorney_hands_up.zip
--- Zipping 'ace_atorney_pointing' ---
✅ Created ace_atorney_pointing.zip
--- Zipping 'think_mark' ---
✅ Created think_mark.zip

📦 Batch Zipping Complete. 3 archive(s) created in your Drive.
Zip files are located in: /content/drive/MyDrive/
